In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import time
import sys

from tensorflow.keras import backend as kb
from resizeimage import resizeimage
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Layer, Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

from IPython.display import clear_output
from tqdm import tqdm

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 8)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)


In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)


tmp = list(data_dir.glob('*/'))
classes = []
for i in tmp:
    if os.path.isdir(i):
        classes.append(os.path.basename(i))
print(classes)

Images = []
for class_ in classes:
    list_ = list(data_dir.glob(class_ + '/*'))
    Images.append(list_)
    

numBatches = 100
numImagesOfClassesForBatch = 4
width = 180
height = 180

inputs_train = np.empty([1, len(classes), numImagesOfClassesForBatch, width, height, 3])
for i in range(int(numBatches * 0.7)):
    
    input_ = np.empty([1, numImagesOfClassesForBatch, width, height, 3])
    
    for class_ in range(len(classes)):
        numClass = len(Images[class_])
        choice_idx = np.random.choice(numClass, numImagesOfClassesForBatch, replace=False)
        choice = [Images[class_][j] for j in choice_idx]
        
        
        buffer = np.empty([1, width, height, 3])
        
        for path in choice:
            with PIL.Image.open(path) as img:
                im = resizeimage.resize_cover(img, [width, height])
                im = np.expand_dims(np.array(im), axis=0)
                buffer = np.concatenate((buffer, im), axis=0)
                
                #print(im.shape)
                #plt.imshow(im)
                
        buffer = np.delete(buffer, 0, axis=0)
        buffer = np.expand_dims(buffer, axis=0)
        
        input_ = np.concatenate((input_, buffer), axis=0)
        
        
        #print(buffer.shape)
        
        
    input_ = np.delete(input_, 0, axis=0)
    input_ = np.expand_dims(input_, axis=0)
    
    inputs_train = np.concatenate((inputs_train, input_), axis=0)
inputs_train = np.delete(inputs_train, 0, axis=0)


inputs_test = np.empty([1, len(classes), numImagesOfClassesForBatch, width, height, 3])
for i in range(int(numBatches * 0.3)):
    
    input_ = np.empty([1, numImagesOfClassesForBatch, width, height, 3])
    
    for class_ in range(len(classes)):
        numClass = len(Images[class_])
        choice_idx = np.random.choice(numClass, numImagesOfClassesForBatch, replace=False)
        choice = [Images[class_][j] for j in choice_idx]
        
        
        buffer = np.empty([1, width, height, 3])
        
        for path in choice:
            with PIL.Image.open(path) as img:
                im = resizeimage.resize_cover(img, [width, height])
                im = np.expand_dims(np.array(im), axis=0)
                buffer = np.concatenate((buffer, im), axis=0)
                
                #print(im.shape)
                #plt.imshow(im)
                
        buffer = np.delete(buffer, 0, axis=0)
        buffer = np.expand_dims(buffer, axis=0)
        
        input_ = np.concatenate((input_, buffer), axis=0)
        
        
        #print(buffer.shape)
        
        
    input_ = np.delete(input_, 0, axis=0)
    input_ = np.expand_dims(input_, axis=0)
    
    inputs_test = np.concatenate((inputs_test, input_), axis=0)
inputs_test = np.delete(inputs_test, 0, axis=0)




print(inputs_train.shape)
print(inputs_test.shape)

In [ ]:
def normalized(x):
    min_ = tf.reduce_min(x)
    max_ = tf.reduce_max(x)
    
    y = (tf.subtract(x, min_)/ tf.subtract(max_, min_))
    return y

def distanceLoss(outputs):
    total_mean = None
    class_means = []
    
    for i in range(outputs.shape[0]):
        output_class = outputs[i,:,:,:,:]
        output_class_mean = tf.math.reduce_mean(output_class, axis=0)
        class_means.append(output_class_mean)
        
        output_class_mean = tf.expand_dims(output_class_mean, axis=0)
        
        if total_mean is None:
            total_mean = output_class_mean
        else:
            total_mean = tf.concat([total_mean, output_class_mean], axis=0)
    
    total_mean = tf.math.reduce_mean(total_mean, axis=0)
    #total_mean = normalized(total_mean)
    
    distance_sum = tf.constant(0, dtype=tf.float32)
    for i in class_means:
        #output_class_mean = normalized(i)
        distance = tf.reduce_sum(tf.abs(tf.subtract(total_mean, i)), [0, 1, 2])
        #distance = tf.reduce_sum(tf.math.multiply(total_mean, output_class_mean), [0, 1, 2])
        distance_sum += distance
    
    #distance_mean /= outputs.shape[0]
    Loss = tf.math.exp(-tf.norm(distance_sum))
    #Loss = 1 / (tf.norm(distance_mean) + kb.epsilon())
    return Loss


def distanceLoss2(outputs):
    
    distance_means = tf.constant(0, dtype=tf.float32)
    
    for i in range(outputs.shape[0]):
        output_class = outputs[i,:,:,:,:]
        output_class_mean = tf.math.reduce_mean(output_class, axis=0)
        #output_class_mean = normalized(output_class_mean)
        
        distance_mean = tf.constant(0, dtype=tf.float32)
        for j in range(output_class.shape[0]):
            single_output = output_class[j,:,:,:]
            #single_output = normalized(single_output)
            #distance = tf.reduce_sum(tf.math.multiply(output_class_mean, single_output), [0, 1, 2])
            distance = tf.reduce_sum(tf.abs(tf.subtract(output_class_mean, single_output)), [0, 1, 2])
            distance_mean += distance
        distance_mean /= output_class.shape[0]
        distance_means += distance_mean
    
    distance_means /= outputs.shape[0]
    Loss = distance_means
    #print(Loss)
    return Loss
            
        
        


In [ ]:
class MyModel(Model):
    def __init__(self, numFeatures=3):
        super(MyModel, self).__init__()
        self.numFeatures = numFeatures
        self.input_shape_ = None
        self.custom_layers = []
        
    def build(self, input_shape):
        self.input_shape_ = input_shape[1:]
        
        self.custom_layers.append(Conv2D(self.numFeatures, 3, input_shape=self.input_shape_))
        self.pooling_ = MaxPool2D()
            
    def addLayer(self, numFeatures):
        self.numFeatures = numFeatures
        shape = tf.convert_to_tensor(self.input_shape_).numpy()
        for i in range(len(self.custom_layers)):
            shape[0] = shape[0] - 2
            shape[1] = shape[1] - 2
            
        self.custom_layers.append(Conv2D(self.numFeatures, 3, input_shape=shape))
        self.custom_layers[-1].build(shape)
        
            
    def addNode(self, numFeatures):
        weights = self.custom_layers[-1].get_weights()
        
        #print(weights[0].shape)
        #print(weights[1].shape)
        
        shape = list(weights[0].shape[:-1])
        shape.append(numFeatures)
        #print(shape)
        
        #shape = np.expand_dims(shape, axis=-1)
        tmp = np.random.normal(size=shape)
        weights[0] = np.concatenate((weights[0], tmp), axis=-1)
        #print(weights[0].shape)
        
        tmp = np.random.normal(size = (numFeatures,))
        weights[1] = np.concatenate((weights[1], tmp))
        
        #print(weights[1].shape)
        shape = tf.convert_to_tensor(self.input_shape_).numpy()
        for i in range(len(self.custom_layers) - 1):
            shape[0] = shape[0] - 2
            shape[1] = shape[1] - 2
            
        self.numFeatures += numFeatures
        self.custom_layers[-1] = Conv2D(self.numFeatures, 3, input_shape=shape)
        self.custom_layers[-1].build(shape)
        self.custom_layers[-1].set_weights(weights)

    def call(self, x):
        #results = {}
        
        for idx_layer, layer_ in enumerate(self.custom_layers):
            x = tf.math.sin(layer_(x))
            x = self.pooling_(x)
        
        """
        shape = list(x.shape[1:])
        shape.insert(0, 1)
        answer = tf.random.normal(shape=shape)
        for i in range(x.shape[0]):
            single_out = x[i,:,:,:]
            single_out_normalized = normalized(single_out)
            single_out_normalized = tf.expand_dims(single_out_normalized, axis=0)
            
            answer = tf.concat([answer, single_out_normalized], axis=0)
        answer = answer[1:,:,:,:]
        """
        return x

model = MyModel(numFeatures=10)

In [ ]:
tmp = tf.random.normal(shape=(4, 180, 180, 3))
answer = model(tmp)

print(answer.shape)
"""
shape = list(answer.shape[1:])
shape.insert(0, 1)
tmp = tf.random.normal(shape=shape)
print(tmp.shape)
"""

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002)
L1_train = tf.keras.metrics.Mean()
L2_train = tf.keras.metrics.Mean()

@tf.function
def train_step(input_):
    with tf.GradientTape() as tape:
        output = None
        for i in range(input_.shape[0]):
            input_oneClass = input_[i,:,:,:,:]
            output_oneClass = model(input_oneClass)
            output_oneClass = tf.expand_dims(output_oneClass, axis=0)
            
            if output == None:
                output = output_oneClass
            else:
                output = tf.concat([output, output_oneClass], axis=0)
        
        l1 = distanceLoss(output)
        l2 = distanceLoss2(output)
        Loss = l1 + l2
        #Loss = distanceLoss2(output)
        
    gradients = tape.gradient(Loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    L1_train(l1)
    L2_train(l2)
    
L1_valid = tf.keras.metrics.Mean()
L2_valid = tf.keras.metrics.Mean()
@tf.function
def test_step(input_):
    output = None
    for i in range(input_.shape[0]):
        input_oneClass = input_[i,:,:,:,:]
        output_oneClass = model(input_oneClass)
        output_oneClass = tf.expand_dims(output_oneClass, axis=0)
        
        if output == None:
            output = output_oneClass
        else:
            output = tf.concat([output, output_oneClass], axis=0)
            
    l1 = distanceLoss(output)
    l2 = distanceLoss2(output)
    L1_valid(l1)
    L2_valid(l2)

In [ ]:
EPOCHS = 20000
patience = 100
stopped_epoch = 0
best_weights = None
best = np.Inf
wait = 0

history_l1_train = []
history_l2_train = []
history_l1_valid = []
history_l2_valid = []


model.addNode(20)
#model.addLayer(20)
start_time = time.time()
for epoch in range(EPOCHS):
    clear_output(wait=True)
    
    L1_train.reset_states()
    L2_train.reset_states()
    for i in range(inputs_train.shape[0]):
        input_ = inputs_train[i,:,:,:,:,:]
        train_step(input_)
    
    L1_valid.reset_states()
    L2_valid.reset_states()
    for i in range(inputs_test.shape[0]):
        input_ = inputs_test[i,:,:,:,:,:]
        test_step(input_)
    
    
    template = '에포크: {}, L1_train: {:.4f}, L2_train: {:.4f}, L1_vaid: {:.4f}, L2_valid: {:.4f}, 걸린 시간: {:.3f}'
    print (template.format(epoch+1,
                         L1_train.result(),
                           L2_train.result(),
                         L1_valid.result(),
                           L2_valid.result(),
                           (time.time() - start_time)))
    
    history_l1_train.append(L1_train.result())
    history_l2_train.append(L2_train.result())
    
    history_l1_valid.append(L1_valid.result())
    history_l2_valid.append(L2_valid.result())
    
    
    if np.less(float(L1_valid.result() + L2_valid.result()), best):
        best = float(L1_valid.result() + L2_valid.result())
        best_weights = model.get_weights()
        wait = 0
    else:
        wait +=1
        if wait >= patience:
            model.set_weights(best_weights)
            stopped_epoch = epoch
            print('Early Stopped !')
            break
    
    
fig, ax = plt.subplots(2, figsize=(18,20))

ax[0].set_title('L1')
x = np.arange(len(history_l1_train))
ax[0].plot(x, history_l1_train, 'r-', label = 'trainLoss')
ax[0].plot(x, history_l1_valid, 'b-', label = 'validLoss')
ax[0].legend()

ax[1].set_title('L2')
ax[1].plot(x, history_l2_train, 'r-', label = 'trainLoss')
ax[1].plot(x, history_l2_valid, 'b-', label = 'validLoss')
ax[1].legend()
fig.tight_layout()
plt.show()

In [ ]:
model.summary()